# Shortest Path:   
## Given Network, find shortest path between two nodes.

###  Bus 36109 "Advanced Decision Modeling with Python", Don Eisenstein
Don Eisenstein &copy; Copyright 2023, University of Chicago 

---

List of Arcs and travel time along the arc 

| From Node |  To Node  |  Travel time 
| ------------- |-----  | ---|
| A    |  B |  4 |  
| A    |  C |  2 | 
| B    |  C |  5 |
| C    |  B |  8 |
| B    |  D | 10 |
| C    |  E |  3 |
| E    |  D |  4 |
| D    |  F |  5 |
| E    |  F | 10 |


Source Node: A

Sink Node: F


In [1]:
from pprint import pprint

In [2]:
nodes = ['A', 'B', 'C', 'D', 'E', 'F']
source_node = 'A' # Where we start
sink_node = 'F' # Where we end

In [3]:
arcs = [
 {'from_node': 'A', 'to_node': 'B', 'cost': 4},
 {'from_node': 'A', 'to_node': 'C', 'cost': 2},
 {'from_node': 'B', 'to_node': 'C', 'cost': 5},
 {'from_node': 'C', 'to_node': 'B', 'cost': 8},
 {'from_node': 'B', 'to_node': 'D', 'cost': 10},
 {'from_node': 'C', 'to_node': 'E', 'cost': 3},
 {'from_node': 'E', 'to_node': 'D', 'cost': 4},
 {'from_node': 'D', 'to_node': 'F', 'cost': 5},
 {'from_node': 'E', 'to_node': 'F', 'cost': 10}
]
pprint(arcs)

[{'cost': 4, 'from_node': 'A', 'to_node': 'B'},
 {'cost': 2, 'from_node': 'A', 'to_node': 'C'},
 {'cost': 5, 'from_node': 'B', 'to_node': 'C'},
 {'cost': 8, 'from_node': 'C', 'to_node': 'B'},
 {'cost': 10, 'from_node': 'B', 'to_node': 'D'},
 {'cost': 3, 'from_node': 'C', 'to_node': 'E'},
 {'cost': 4, 'from_node': 'E', 'to_node': 'D'},
 {'cost': 5, 'from_node': 'D', 'to_node': 'F'},
 {'cost': 10, 'from_node': 'E', 'to_node': 'F'}]


In [4]:
# === SETUP ===
import pulp
import os

# Portable solver setup, to allow work locally (ARM64 architecture) as well as in JupyterHub
from pulp import COIN_CMD
if os.path.exists('/opt/homebrew/opt/cbc/bin/cbc'):
    solver = COIN_CMD(path='/opt/homebrew/opt/cbc/bin/cbc', msg=0)
else:
    solver = pulp.PULP_CBC_CMD(msg=0)

In [5]:
model = pulp.LpProblem("Uber_Assignment",pulp.LpMinimize)

In [6]:
X = {}

for arc in arcs:
    variable_name = f"X_{arc['from_node']}_{arc['to_node']}"
    X[ (arc['from_node'], arc['to_node']) ] = pulp.LpVariable(variable_name, cat = 'Binary')

pprint(X)

{('A', 'B'): X_A_B,
 ('A', 'C'): X_A_C,
 ('B', 'C'): X_B_C,
 ('B', 'D'): X_B_D,
 ('C', 'B'): X_C_B,
 ('C', 'E'): X_C_E,
 ('D', 'F'): X_D_F,
 ('E', 'D'): X_E_D,
 ('E', 'F'): X_E_F}


In [7]:
obj = None
for arc in arcs:
    obj += arc['cost'] * X[ (arc['from_node'], arc['to_node']) ]

model += obj, 'Cost'
print(model)

Uber_Assignment:
MINIMIZE
4*X_A_B + 2*X_A_C + 5*X_B_C + 10*X_B_D + 8*X_C_B + 3*X_C_E + 5*X_D_F + 4*X_E_D + 10*X_E_F + 0
VARIABLES
0 <= X_A_B <= 1 Integer
0 <= X_A_C <= 1 Integer
0 <= X_B_C <= 1 Integer
0 <= X_B_D <= 1 Integer
0 <= X_C_B <= 1 Integer
0 <= X_C_E <= 1 Integer
0 <= X_D_F <= 1 Integer
0 <= X_E_D <= 1 Integer
0 <= X_E_F <= 1 Integer



In [8]:
# Set source node constraint
const = None
for arc in arcs:
    if arc['from_node'] == source_node:
        const += X[ (arc['from_node'], arc['to_node']) ]

model += const == 1, f'source_node_{source_node}'

In [9]:
# Set sink node constraint
const = None
for arc in arcs:
    if arc['to_node'] == sink_node:
        const += X[ (arc['from_node'], arc['to_node']) ]

model += const == 1, f'sink_node_{sink_node}'

Uber_Assignment:
MINIMIZE
4*X_A_B + 2*X_A_C + 5*X_B_C + 10*X_B_D + 8*X_C_B + 3*X_C_E + 5*X_D_F + 4*X_E_D + 10*X_E_F + 0
SUBJECT TO
source_node_A: X_A_B + X_A_C = 1

sink_node_F: X_D_F + X_E_F = 1

VARIABLES
0 <= X_A_B <= 1 Integer
0 <= X_A_C <= 1 Integer
0 <= X_B_C <= 1 Integer
0 <= X_B_D <= 1 Integer
0 <= X_C_B <= 1 Integer
0 <= X_C_E <= 1 Integer
0 <= X_D_F <= 1 Integer
0 <= X_E_D <= 1 Integer
0 <= X_E_F <= 1 Integer



In [10]:
for node in nodes:
    if (node != source_node and node != sink_node):
        const = None
        for arc in arcs:
            # If arc is leaving node
            if arc['from_node'] == node:
                const += X[ (arc['from_node'], arc['to_node']) ]
            # And if arc is entering node
            if arc['to_node'] == node:
                const += -X[ (arc['from_node'], arc['to_node']) ]
        model += const == 0, f'transhipment_{node}'

Uber_Assignment:
MINIMIZE
4*X_A_B + 2*X_A_C + 5*X_B_C + 10*X_B_D + 8*X_C_B + 3*X_C_E + 5*X_D_F + 4*X_E_D + 10*X_E_F + 0
SUBJECT TO
source_node_A: X_A_B + X_A_C = 1

sink_node_F: X_D_F + X_E_F = 1

transhipment_B: - X_A_B + X_B_C + X_B_D - X_C_B = 0

transhipment_C: - X_A_C - X_B_C + X_C_B + X_C_E = 0

transhipment_D: - X_B_D + X_D_F - X_E_D = 0

transhipment_E: - X_C_E + X_E_D + X_E_F = 0

VARIABLES
0 <= X_A_B <= 1 Integer
0 <= X_A_C <= 1 Integer
0 <= X_B_C <= 1 Integer
0 <= X_B_D <= 1 Integer
0 <= X_C_B <= 1 Integer
0 <= X_C_E <= 1 Integer
0 <= X_D_F <= 1 Integer
0 <= X_E_D <= 1 Integer
0 <= X_E_F <= 1 Integer



In [12]:
model.solve(solver)
print(f"Status: {pulp.LpStatus[model.status]}")
print(f"Total travel time = {pulp.value(model.objective)}")
for v in model.variables():
    print(f"{v.name} = {v.varValue}")

Status: Optimal
Total travel time = 14.0
X_A_B = 0.0
X_A_C = 1.0
X_B_C = 0.0
X_B_D = 0.0
X_C_B = 0.0
X_C_E = 1.0
X_D_F = 1.0
X_E_D = 1.0
X_E_F = 0.0
